# Arco-IT GmbH MQP User Interface

## This user interface will allow the user to:

1. Import a data file to be analyzed (CSV)

    1a. Have the user select how many rows they wish to see
    
    1b. Have the user select which rows they would like to omit
2. Create tags, and application data for rule generation
3. Generate Rules
4. Approve, Deny, or modify generated rules


**Welcome to the Arco IT GmbH Official User Interface!**

**--------------------------------------------------------**

**To proceed, please upload a CSV file using the button provided to you.**

In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

#This is where the file upload button will be created
upload = widgets.FileUpload(
    description = 'Upload CSV',
    multiple = False,
    accept = '.csv'
)

#create upper buttons
devices_button = widgets.Button(description = 'Devices')
classifications_button = widgets.Button(description = 'Classifications')
applications_button = widgets.Button(description = 'Applications')
rules_button = widgets.Button(description = 'Rules')

#display buttons - in a horizontal fashion
display(widgets.HBox([widgets.VBox([upload])]))
#                      widgets.VBox([devices_button]),
#                      widgets.VBox([classifications_button]),
#                      widgets.VBox([applications_button]),
#                      widgets.VBox([rules_button])]))


In [2]:
#this takes the resulting dictionary and assigns it to a variable 
uploaded_file_val = upload.value
if(len(uploaded_file_val) == 0):
    raise Exception("You must input a file")
#show that variable
#uploaded_file_val

In [3]:
#access whats important
file_name = next(iter(uploaded_file_val))
#we get the first element in the dictionary
uploaded_file_content = uploaded_file_val[file_name]['content']
#uploaded_file_content

In [4]:
#This gets the number of rows within the CSV
with open('inventory_data.csv') as counter:
    row_counter_with_header = sum(1 for line in counter)
row_counter_without_header = row_counter_with_header-1

In [5]:
import codecs
#decodes from bytes to list and splits the data accordingly
data = uploaded_file_content.decode('utf-8').splitlines()
#below line removes quotes from decoding by iterating through list
data = [i.replace('"', ' ') for i in data]
#data

In [6]:
import csv
import re

#This creates a csv file within the directory that has the original data
with open("inventory_data.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter = '\t')
    for line in data:
        writer.writerow(re.split('\s+,',line))

**There will be an interactive tab system that is configured for you to use. Each tab represents options for the desired mode, and has tools for those modes.**

In [7]:
#----------------------Device Setup-----------------
#This is a slider to select how many rows would like to be displayed
slider_result = widgets.IntSlider(
    value=1,
    min=1,
    max=row_counter_without_header, #we have to subtract here to include the header
    step=1,
    description='Rows:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
link_text = widgets.IntText()
widgets.jslink((slider_result, 'value'), (link_text, 'value'))
#Creates the button for showing resulting CSV
show_csv_button = widgets.Button(description = 'Show CSV')

device_original_output_area = widgets.Output()
device_dropped_output_area = widgets.Output()

#Creates the text box for row omition
deny_list_input = widgets.Textarea(placeholder = "Separate the row values with commas",
                        description = "Rows: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='40px'))#auto adjusts size
    
#Creates the row drop button
drop_rows_button = widgets.Button(description = 'Drop Rows')

#generate the button for the graph
asset_connection_button = widgets.Button(description = "Generate Asset Connection Graph",
                                        layout = widgets.Layout(width='auto', height='40px'))
#This needs to be put somewhere else
#Whats happening is at this point all of the buttons become activated but we need to rerun everything
#to get the updated CSV

device_accordion_one = widgets.Accordion(children=[widgets.HBox([widgets.VBox([slider_result]), widgets.VBox([link_text])])])
device_accordion_one.set_title(0, 'Slider For Selecting How Many Rows to View At Once')
device_accordion_one

#-----------------Classification Setup-----------------------------
#Creates the text box for IP Prefix
ip_prefix_area = widgets.Textarea(placeholder = "Input the IP Prefix",
                        description = "IP Prefix: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='auto'))#auto adjusts size
ip_values = [] #will be used to keep a store of ip's
#Creates the text box for Tag Name
tag_name_area = widgets.Textarea(placeholder = "Input the Tag Name",
                        description = "Name: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='auto'))#auto adjusts size
#Creates the text box for Tag Description
tag_description_area = widgets.Textarea(placeholder = "Input the Tag Description",
                        description = "Description: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='auto'))#auto adjusts size

tag_output_area = widgets.Output()
row_tag_list =[]
#Creates the row add button
add_tag = widgets.Button(description = 'Add')

classification_accordion_one = widgets.Accordion(children=[widgets.HBox([widgets.VBox([ip_prefix_area]),
                                                                                widgets.VBox([tag_name_area]),
                                                                                widgets.VBox([tag_description_area])])])

classification_accordion_one.set_title(0, 'Text Boxes for Tag Configuration')
classification_accordion_one
#-----------------Application Setup-----------------------------
#Creates the text box for Port
port_area = widgets.Textarea(placeholder = "Input the Port Number",
                        description = "Port: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='auto'))#auto adjusts size
#Creates the dropdown for protocol
protocol_dropdown = widgets.Dropdown(options = ['TCP', 'UDP'],
                                     description = 'Protocol',
                                    layout = widgets.Layout(width='auto', height='auto'))
                                     
#Creates the text box for App Name
app_name_area = widgets.Textarea(placeholder = "Input the Application Name",
                        description = "Name: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='auto'))#auto adjusts size
#Creates the text box for App Description
app_description_area = widgets.Textarea(placeholder = "Input the Application Description",
                        description = "Description: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='auto'))#auto adjusts size
app_output_area = widgets.Output()
row_app_list =[]
#Creates the row add button
add_app = widgets.Button(description = 'Add')

application_accordion_one = widgets.Accordion(children=[widgets.HBox([widgets.VBox([port_area]),
                                                                      widgets.VBox([protocol_dropdown]),
                                                                                widgets.VBox([app_name_area]),
                                                                                widgets.VBox([app_description_area])])])

application_accordion_one.set_title(0, 'Text Boxes for Application Configuration')
application_accordion_one

#-----------------Rule Setup-----------------------------
#Creates the text box for Port
order_num = widgets.Textarea(placeholder = "Input the Order Preference",
                        description = "Order: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='auto'))#auto adjusts size
#Creates the dropdown for protocol
application_name = widgets.Dropdown(options = ['SSH', 'Machine Control'],
                                     description = 'Application',
                                    layout = widgets.Layout(width='auto', height='auto'))
                                     
#Creates the text box for Source Area
source_text = widgets.Textarea(placeholder = "Input the Source",
                        description = "Source: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='auto'))#auto adjusts size
#Creates the text box for Target Area
target_area = widgets.Textarea(placeholder = "Input the Target",
                        description = "Target: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='auto'))#auto adjusts size
#Creates the text box for Decision Dropdown
decision_dropdown = widgets.Dropdown(options = ['Allow', 'Disallow'],
                                     description = 'Decision',
                                    layout = widgets.Layout(width='auto', height='auto'))
#Creates the text box for Description Area
description_area = widgets.Textarea(placeholder = "Input the Description",
                        description = "Description: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='auto'))#auto adjusts size
rule_output_area = widgets.Output()
row_rule_list =[]
#Creates the row add button for rule
add_rule = widgets.Button(description = 'Add')
#Creates the row add button for rule (ML generated)
add_ML_rule = widgets.Button(description = 'Auto Generate', layout = widgets.Layout(width='auto', height='auto'))

left_box = widgets.HBox([order_num, application_name, source_text])
right_box = widgets.HBox([target_area, decision_dropdown, description_area])

rule_accordion_one = widgets.Accordion(children=[widgets.VBox([left_box, right_box])])

rule_accordion_one.set_title(0, 'Text Boxes for Rule Configuration')
rule_accordion_one

#------------------Phase 1 Tab-----------------------------------

tab_nest = widgets.Tab()
#The line right below this one has placeholders
tab_nest.children = [device_accordion_one, classification_accordion_one, application_accordion_one, rule_accordion_one]
tab_nest.set_title(0, 'Device Options')
tab_nest.set_title(1, 'Classification Options')
tab_nest.set_title(2, 'Application Options')
tab_nest.set_title(3, 'Rule Options')
tab_nest


In [8]:
def number_devices(iprange):
    ##first take in the IP range
    df= pd.read_csv(r'inventory_data.csv')
    check= re.sub('/',"",iprange)
    min = re.sub('\/.*$',"",iprange)
    min = re.sub('.\\d{0,3}.\\d{0,3}\\d{0,3}\\.\\d{0,3}.',"",min)
    max=re.sub('^.*?/', '', iprange)

    if int(min)>int(max) or int(max)>255 or int(min)<0:
        print("Invalid. Range can only be from 0-255")
        return

    check = re.sub('\\.\\d+$',"",check)
    List_devices= []
    List_IPs=[]
    FinalList_all = []
    count=0
    for (columnName, columnData) in df.iteritems():
        
        if('Address' in columnName):
            for i in range(len(df[columnName])):
                if check in df[columnName].iloc[i] and len(check)==len(re.sub('\\.\\d+$',"",df[columnName].iloc[i])):
                    curr_val= re.sub('.\\d{1,3}.\\d{1,3}\\d{1,3}\\.\\d{1,3}.',"",df[columnName].iloc[i])
                    
                    if int(curr_val)>=int(min) and int(curr_val)<=int(max):
                        
                        FinalList_all.append(df[columnName].iloc[i])
                        if(df[columnName].iloc[i] not in List_IPs):
                            List_IPs.append(df[columnName].iloc[i])
                        
                        for(name,data)in df.iteritems():
                            if('Device' in name) :
                                count+=1
                                if(df[name].iloc[i] not in List_devices):
                                    List_devices.append(df[name].iloc[i])

    return len(FinalList_all)
#number_devices(ip_prefix_area.value)

In [9]:
#------------Creates the CSV for tags------------------
from IPython.display import display, clear_output
header = ['IP-Prefixes', 'Name', 'Description', 'Matched Devices']

if len(ip_prefix_area.value) != 0:
    row_tag_list.append([ip_prefix_area.value, tag_name_area.value, tag_description_area.value,number_devices(ip_prefix_area.value)])
    
#--Tag CSV--   
def add_tag_row(a):
    with open("tag_classifications.csv", "w", newline='') as f:
        tag_writer = csv.writer(f, delimiter=',')
        tag_writer.writerow(header) # write header
        for lines in row_tag_list:
            tag_writer.writerow(lines)
    with tag_output_area:
        clear_output()
        tag_classification_csv = pd.read_csv('tag_classifications.csv')
        display(tag_classification_csv)
#----button click----
add_tag.on_click(add_tag_row)

In [10]:
#------------Creates the CSV for applications------------------
app_header = ['Port', 'Protocol', 'Name', 'Description']

row_app_list.append([port_area.value, protocol_dropdown.value, app_name_area.value, app_description_area.value])

add_app.click_count = 0

#--App CSV--
def add_app_row(a):
    
    add_app.click_count += 1
#    --Create an output area here --
    
    with open("applications.csv", "w", newline='') as f:
        app_writer = csv.writer(f, delimiter=',')
        app_writer.writerow(app_header) # write header
        for lines in row_app_list:
            app_writer.writerow(lines)
    with app_output_area:
        clear_output()
        applications_csv = pd.read_csv('applications.csv')
        display(applications_csv)
#----button click----
add_app.on_click(add_app_row)

In [11]:
#------------Creates the CSV for rules------------------
from IPython.display import display, clear_output

app_header = ['Order', 'Application', 'Source', 'Target', 'Decision', 'Name']

row_rule_list.append([order_num.value, application_name.value, source_text.value, target_area.value,
                     decision_dropdown.value, description_area.value])

#--Rule CSV--
def add_rule_row(a):    
    with open("rules.csv", "w", newline='') as f:
        rule_writer = csv.writer(f, delimiter=',')
        rule_writer.writerow(app_header) # write header
        for lines in row_rule_list:
            rule_writer.writerow(lines)
    with rule_output_area:
        clear_output()
        rule_csv = pd.read_csv('rules.csv')
        display(rule_csv)
#----button click----
add_rule.on_click(add_rule_row)

In [12]:
#------------Creates the CSV for Machine Learning rules------------------

In [13]:
original_csv = pd.read_csv('inventory_data.csv', skiprows = range(1,(row_counter_with_header - slider_result.value)))

def show_csv(b):
    with device_original_output_area:
        clear_output()
        display(original_csv)

show_csv_button.on_click(show_csv)
#print("Use the slider above to select how many rows of data you would like to see then click on the devices button up top.")

In [14]:
#display(drop_rows_button)
def drop_rows(a):
    if(deny_list[0]==''):
        raise Exception("This button cannot be used if there is no text input.")
    #drop the selected rows from the dataset
    with device_dropped_output_area:
        clear_output()
        original_csv.drop(original_csv.index[deny_list_ints], inplace = True)
        display(original_csv)

drop_rows_button.on_click(drop_rows)

In [15]:
#This creates another csv file within the directory that has the modified data
with open("inventory_data.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter = '\t')
    for line in data:
        writer.writerow(re.split('\s+,',line))

In [29]:
#Imports for connection graph
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.axes._axes import _log as matplotlib_axes_logger

#display(asset_connection_button)

def generate_asset_connection_graph(b):
    matplotlib_axes_logger.setLevel('ERROR')

    common_port_cutoff = 1024

    flow_data = pd.read_csv('sample_data/sample_data.csv', header=0)
    ip_data = pd.read_csv('sample_data/ip_inventory.csv', header=0, index_col=0)

    def map_ips_to_col(address_name, col_name):
        site_list = []
        for ip in flow_data[address_name]:
            try:
                name = ip_data.at[ip, col_name]
                site_list.append(name)
            except:
                site_list.append(pd.NaT)
        return pd.DataFrame(site_list, columns=[address_name])

    site_a = map_ips_to_col('Address A', 'Plant Area')
    site_b = map_ips_to_col('Address B', 'Plant Area')
    port_a = flow_data['Port A']
    port_b = flow_data['Port B']

    data = pd.concat([site_a, port_a, site_b, port_b], 1)
    data = data.dropna()

    #print('Data loaded')

    main_graph = {}
    for i, d in data.iterrows():
        ports = []
        if d['Port A'] <= common_port_cutoff:
            ports.append(d['Port A'])
        if d['Port B'] <= common_port_cutoff:
            ports.append(d['Port B'])
        if d['Address A'] in main_graph:
            connections = main_graph[d['Address A']]
            if d['Address B'] in connections:
                used_ports = connections[d['Address B']]
                for p in ports:
                    if not p in used_ports:
                        used_ports.append(p)
            elif len(ports) > 0:
                connections[d['Address B']] = ports
        elif len(ports) > 0:
            main_graph[d['Address A']] = {d['Address B']: ports}

    #print('Main graph created')

    inter_graph = {}
    ip_a = flow_data['Address A']
    ip_b = flow_data['Address B']
    for i in range(site_a.shape[0]):
        site_a_i = site_a['Address A'][i]
        ports = []
        if port_a[i] <= common_port_cutoff:
            ports.append(port_a[i])
        if port_b[i] <= common_port_cutoff:
            ports.append(port_b[i])
        if site_a_i == site_b['Address B'][i]:
            ip_a_i = ip_a[i]
            ip_b_i = ip_b[i]
            if site_a_i in inter_graph:
                if ip_a_i in inter_graph[site_a_i]:
                    if ip_b_i in inter_graph[site_a_i][ip_a_i]:
                        used_ports = inter_graph[site_a_i][ip_a_i][ip_b_i]
                        for p in ports:
                            if not p in used_ports:
                                used_ports.append(p)
                    elif len(ports) > 0:
                        inter_graph[site_a_i][ip_a_i][ip_b_i] = ports
                elif len(ports) > 0:
                    inter_graph[site_a_i][ip_a_i] = {ip_b_i: ports}
            elif len(ports) > 0:
                inter_graph[site_a_i] = {ip_a_i: {ip_b_i: ports}}

    #print('Internal graphs created')

    def formate_node_label(label):
        if label == 'Other ProcN Devices':
            return 'Other ProcN\n Devices'
        if label == 'EAF Dust Washing':
            return 'EAF Dust\nWashing'
        return label

    def plot_graph(graph_name, graph, fig_size):
        vis_graph = nx.DiGraph()
        edge_labels = {}
        for k0, v0 in graph.items():
            for k1, v1 in v0.items():
                if k0 != k1:
                    if len(v1) > 0:
                        vis_graph.add_edge(k1, k0)
                        edge_labels[(k0, k1)] = ', '.join(map(str, v1))

        pos = nx.kamada_kawai_layout(vis_graph)
        fig = plt.figure(figsize=fig_size, dpi=300)
        nx.draw(vis_graph, pos, node_size=5500, node_color=[0.9,0.9,0.9], labels={node:formate_node_label(str(node)) for node in vis_graph.nodes()}, font_size=10, edgecolors='black')
        nx.draw_networkx_edge_labels(vis_graph, pos, edge_labels=edge_labels)

        plt.savefig('output/asset_connection_' + graph_name + '.png')

    plot_graph('Main', main_graph, (9, 9))
    #print('Plotted main graph')

    for site, connections in inter_graph.items():
        plot_graph(site, connections, (7, 7))
        #print('Plotted ' + site + ' internal graph')
    
asset_connection_button.on_click(generate_asset_connection_graph)


**There have been a few more tools generated:**

 **- A button to show the resulting CSV file.**
 
 **- A text box to insert rows that you would like to remove.** 
 
Keep in mind, the text box numbers have to be separated with commas and they follow 0 based indexxing. So, typing 0 into the box aims to eliminate the first row of the CSV, typing 1 aims for the second row, so on and so fourth.

In [30]:
#------------------Device Setup----------
device_accordion_two = widgets.Accordion(children=[show_csv_button, device_original_output_area,
                                                   deny_list_input])
device_accordion_two.set_title(0, 'Button To Show the Resulting CSV')
device_accordion_two.set_title(1, 'Device CSV Output Area')
device_accordion_two.set_title(2, 'Text Box To Remove Rows')
device_accordion_two

#------------------Classification Setup----------
classification_accordion_two = widgets.Accordion(children=[add_tag, tag_output_area])

classification_accordion_two.set_title(0, 'Button to Add Tag and Show Resulting CSV')
classification_accordion_two.set_title(1, 'Classification CSV Output Area')
classification_accordion_two
tab_nest = widgets.Tab()
#------------------Application Setup----------
application_accordion_two = widgets.Accordion(children=[add_app, app_output_area])

application_accordion_two.set_title(0, 'Button to Add Application and Show Resulting CSV')
application_accordion_two.set_title(1, 'Application CSV Output Area')
application_accordion_two
#------------------Rule Setup----------
rule_accordion_two = widgets.Accordion(children=[add_rule,rule_output_area, add_ML_rule])

rule_accordion_two.set_title(0, 'Button to Add Rule and Show Resulting CSV')
rule_accordion_two.set_title(1, 'Created Rule CSV Output Area')
rule_accordion_two.set_title(2, 'Button to Add Machine Learning Rules and Show Resulting CSV')
rule_accordion_two.set_title(3, 'Machine Learning Generated CSV Output Area')
rule_accordion_two
#-----------------Tab Creation----------------
tab_nest = widgets.Tab()
#The line right below this one has placeholders
tab_nest.children = [device_accordion_two, classification_accordion_two, application_accordion_two, rule_accordion_two]
tab_nest.set_title(0, 'Device Options')
tab_nest.set_title(1, 'Classification Options')
tab_nest.set_title(2, 'Application Options')
tab_nest.set_title(3, 'Rule Options')
tab_nest

In [31]:
if(deny_list_input.value!=''):
    #separate variables
    deny_list = deny_list_input.value.split(",")
    #convert into integers
    deny_list_ints = [int(i) for i in deny_list]
    #deny_list_ints #this just shows what the contents are
else:
    deny_list = deny_list_input.value.split(",")
    deny_list_ints =[0]

**This is the final wave for the device side of things. There a few things here:**

 **- A button to drop the selected rows from the previous step**
 
 **- A button to generate an asset connection graph**

In [32]:
#because the buttons are created before, the accordion comes after so it can do something
device_accordion_three = widgets.Accordion(children=[drop_rows_button, device_original_output_area,
                                                     asset_connection_button])
device_accordion_three.set_title(0, 'Button to Drop Rows and Show Resulting CSV')
device_accordion_three.set_title(1, 'Dropped Device CSV Output Area')
device_accordion_three.set_title(2, 'Button for Generating Asset Graph')
device_accordion_three.set_title(3, 'Resulting Asset Graph Area')
device_accordion_three
tab_nest = widgets.Tab()
#The line right below this one has placeholders
tab_nest.children = [device_accordion_three]#, device_accordion_three, device_accordion_three, device_accordion_three]
tab_nest.set_title(0, 'Device Options')
#tab_nest.set_title(1, 'Classification Options')
#tab_nest.set_title(2, 'Application Options')
#tab_nest.set_title(3, 'Rule Options')
tab_nest